In [1]:
import os
print(os.getcwd())   
print("Current working directory:", os.getcwd())
print("Files here:", os.listdir())
if "data" in os.listdir():
    print("Files inside ./data:", os.listdir("data"))
working_directory = os.getcwd()
print(working_directory)

import pandas as pd
df = pd.read_csv("data/survey_results.csv")
print(df.head())


/drive
Current working directory: /drive
Files here: ['Untitled10.ipynb', 'Untitled11.ipynb', 'Untitled5.ipynb', 'Untitled6.ipynb', 'Untitled7.ipynb', 'Untitled8.ipynb', 'Untitled9.ipynb', 'cleaned_survey_results.csv', 'survey_cleaned.csv', 'README.md', 'data', 'notebooks']
Files inside ./data: ['Untitled12.ipynb', 'survey_results.csv', 'audio.wav', 'iris.csv', 'marie.png', 'matplotlib.png', 'Museums_in_DC.geojson', 'bar.vl.json', 'fasta-example.fasta']
/drive
   Respondent                                         MainBranch Hobbyist  \
0           1                     I am a developer by profession      Yes   
1           2                     I am a developer by profession       No   
2           3                        I code primarily as a hobby      Yes   
3           4                     I am a developer by profession      Yes   
4           5  I used to be a developer by profession, but no...      Yes   

    Age Age1stCode CompFreq  CompTotal  ConvertedComp             Countr

In [14]:
import pandas as pd
import re

df.head()
df.shape        
df.columns[:30] 
demographics = ["Country", "Age", "EdLevel"]
opinions = ["MiscTechDesireNextYear", "PlatformDesireNextYear", "Hobbyist", "WorkWeekHrs"]

df_clean = df.copy()
 
# Make a copy for cleaning
df_clean = df_clean.dropna(subset=["Country", "Age"])


df_small = df[["Country", "Age", "EdLevel", "MiscTechDesireNextYear", "PlatformDesireNextYear", "Hobbyist", "WorkWeekHrs",]]

# Now it's safe to clean df_clean
df_clean = df_clean.dropna(subset=["Country", "Age"])

df_clean["EdLevel"] = df_clean["EdLevel"].fillna("Other/NA")

# Apply  cleaning 
print(df_clean[["Country", "Age", "EdLevel"]].head())

df_small = df[demographics + opinions].copy()
df_small.head() #lots of NAs here, need to remove 
df_clean = df_clean.dropna(subset=["Country", "Age"])
df_clean["EdLevel"] = df_clean["EdLevel"].fillna("Other/NA")


#print(df["EdLevel"].unique())
def clean_edlevel(x):
    if pd.isnull(x):
        return "Other"
    x = str(x).lower()
    if re.search("bachelor",x):
        return "Bachelor's Degree"
    elif re.search("master", x):
        return "Master's Degree"
    elif re.search("ph.d.|doctor", x):
        return "Doctoral degree"
    elif re.search("associate|college|secondary|no formal", x):
        return "Less than Bachelor’s"
    else:
         return "Other"
df_clean["EdLevel"] = df_clean["EdLevel"].apply(clean_edlevel)

for col in ["MiscTechDesireNextYear", "PlatformDesireNextYear", "Hobbyist", "WorkWeekHrs"]:
    df_clean[col] = df_clean[col].fillna("NA")
df_clean.head()

#grouping and organizing data
df_clean["AgeGroup"] = pd.cut(
    df_clean["Age"],
    bins=[0,25,35,45,55,65,100], #100 upper bound
    labels=["<=24","25-35","36-45","46-55","56-66","65+"]
)
df_clean[["Age", "AgeGroup"]].head(20)

df_small = df[demographics + opinions].copy()


#organizing the different education levels 

df["clean_edlevel"] = df["EdLevel"].apply(clean_edlevel)
df_clean["EdLevel"] = df_clean["EdLevel"].apply(clean_edlevel)


        
print(clean_edlevel("Bachelor’s Degree in CS"))
print(clean_edlevel("Ph.D. in Statistics"))
print(clean_edlevel("some random string"))

df["clean_edlevel"] = df["EdLevel"].apply(clean_edlevel)
print(df[["EdLevel", "clean_edlevel"]].head()) 

df = df.rename(columns={
    "clean_edlevel": "Education Level",
    "AgeGroup": "Age Group",
    "WorkWeekHrs": "Work Week Hours",
    "MiscTechDesireNextYear": "Tech Desired Next Year",
    "PlatformDesireNextYear": "Platform Desired Next Year",
    "Hobbyist": "Hobbyist Coder"
})
#df= df.drop(columns=["Education Level"])


          Country   Age                                          EdLevel
3         Albania  25.0  Master’s degree (M.A., M.S., M.Eng., MBA, etc.)
4   United States  31.0     Bachelor’s degree (B.A., B.S., B.Eng., etc.)
7   United States  36.0     Bachelor’s degree (B.A., B.S., B.Eng., etc.)
8         Tunisia  30.0               Professional degree (JD, MD, etc.)
9  United Kingdom  22.0  Master’s degree (M.A., M.S., M.Eng., MBA, etc.)
Bachelor's Degree
Doctoral degree
Other
                                           EdLevel      clean_edlevel
0  Master’s degree (M.A., M.S., M.Eng., MBA, etc.)    Master's Degree
1     Bachelor’s degree (B.A., B.S., B.Eng., etc.)  Bachelor's Degree
2                                              NaN              Other
3  Master’s degree (M.A., M.S., M.Eng., MBA, etc.)    Master's Degree
4     Bachelor’s degree (B.A., B.S., B.Eng., etc.)  Bachelor's Degree


In [15]:
df.isnull().any()

Respondent         False
MainBranch          True
Hobbyist Coder      True
Age                 True
Age1stCode          True
                   ...  
WelcomeChange       True
Work Week Hours     True
YearsCode           True
YearsCodePro        True
Education Level    False
Length: 62, dtype: bool

In [16]:
df[df["MainBranch"].isna()==1]

,Respondent,MainBranch,Hobbyist Coder,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,Work Week Hours,YearsCode,YearsCodePro,Education Level
64162,90,NaN,Yes,24.0,18,NaN,NaN,NaN,Serbia,NaN,...,Appropriate in length,No,"Computer science, computer engineering, or sof...",React.js,Express;Gatsby;React.js;Vue.js,Just as welcome now as I felt last year,NaN,4,NaN,Bachelor's Degree
64163,889,NaN,Yes,NaN,14,NaN,NaN,NaN,United States,NaN,...,Appropriate in length,No,NaN,Angular.js;jQuery;React.js;Vue.js,NaN,A lot less welcome now than last year,NaN,5,Less than 1 year,Less than Bachelor’s
64164,909,NaN,Yes,NaN,13,NaN,NaN,NaN,United Kingdom,NaN,...,Appropriate in length,No,NaN,NaN,NaN,Just as welcome now as I felt last year,NaN,10,Less than 1 year,Other
64165,1173,NaN,Yes,NaN,NaN,NaN,NaN,NaN,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
64166,1264,NaN,Yes,NaN,16,NaN,NaN,NaN,Turkey,NaN,...,Appropriate in length,NaN,NaN,NaN,Laravel,A lot less welcome now than last year,NaN,30,NaN,Doctoral degree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64456,64858,NaN,Yes,NaN,16,NaN,NaN,NaN,United States,NaN,...,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,NaN,NaN,10,Less than 1 year,Master's Degree
64457,64867,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Morocco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
64458,64898,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Viet Nam,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
64459,64925,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Poland,NaN,...,NaN,NaN,NaN,Angular;Angular.js;React.js,NaN,NaN,NaN,NaN,NaN,Other


In [20]:
df["MainBranch"] = df["MainBranch"].fillna(0)
df["Hobbyist"]= df["Hobbyist"].fillna(0)
df["Age"]= df["Age"].fillna(0)
df["Age1stCode"]= df["Age1stCode"].fillna(0)
df["WebframeWorkedWith"]= df["WebframeWorkedWith"].fillna(0)
df["WelcomeChange"]= df["WelcomeChange"].fillna(0)
df["WorkWeekHrs"]= df["WorkWeekHrs"].fillna(0)
df["YearsCode"]= df["YearsCode"].fillna(0)
df["YearsCodePro"]= df["YearsCodePro"].fillna(0)
df["CompFreq"]= df["CompFreq"].fillna(0)
df["CompTotal"]= df["CompTotal"].fillna(0)
df["ConvertedComp"]= df["ConvertedComp"].fillna(0)  
df["CurrencyDesc"]= df["CurrencyDesc"].fillna(0)


#could do fill.na(unknown)for qualitative variables such as main branch 

df[df["Respondent"] == 90]
# to check if the code above worked for an example respondent
#df.rename({"MiscTechDesireNextYear":"DesiredTech_NextYr"},axis = 1)

<class 'KeyError'>: 'Hobbyist'

In [18]:
#organizing the different education levels 
import pandas as pd
import re

df["clean_edlevel"] = df["EdLevel"].apply(clean_edlevel)
df_clean["EdLevel"] = df_clean["EdLevel"].apply(clean_edlevel)

#print(df["EdLevel"].unique())
def clean_edlevel(x):
    if pd.isnull(x):
        return "Other"
    x = str(x).lower()
    if re.search("bachelor",x):
        return "Bachelor's Degree"
    elif re.search("master", x):
        return "Master's Degree"
    elif re.search("ph.d.|doctor", x):
        return "Doctoral degree"
    elif re.search("associate|college|secondary|no formal", x):
        return "Less than Bachelor’s"
    else:
         return "Other"
        
print(clean_edlevel("Bachelor’s Degree in CS"))
print(clean_edlevel("Ph.D. in Statistics"))
print(clean_edlevel("some random string"))

df["clean_edlevel"] = df["EdLevel"].apply(clean_edlevel)
print(df[["EdLevel", "clean_edlevel"]].head()) 


df = df.rename(columns={
    "clean_edlevel": "Education Level",
    "AgeGroup": "Age Group",
    "WorkWeekHrs": "Work Week Hours",
    "MiscTechDesireNextYear": "Tech Desired Next Year",
    "PlatformDesireNextYear": "Platform Desired Next Year",
    "Hobbyist": "Hobbyist Coder"
})
#df= df.drop(columns=["Education Level"])

Bachelor's Degree
Doctoral degree
Other
                                           EdLevel      clean_edlevel
0  Master’s degree (M.A., M.S., M.Eng., MBA, etc.)    Master's Degree
1     Bachelor’s degree (B.A., B.S., B.Eng., etc.)  Bachelor's Degree
2                                              NaN              Other
3  Master’s degree (M.A., M.S., M.Eng., MBA, etc.)    Master's Degree
4     Bachelor’s degree (B.A., B.S., B.Eng., etc.)  Bachelor's Degree


In [ ]:
print(df["clean_edlevel"].value_counts())

In [ ]:
df.to_csv("cleaned_file.csv", index=False, encoding='utf-8')